### Import Library

In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
from sklearn.model_selection import train_test_split
from collections import Counter
import os
import re
import ast
import uuid
import shutil

c:\Users\Sev\miniconda3\envs\tfenv\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


### Preprocessing Data

In [2]:
recipe_df = pd.read_csv("Food.csv", nrows=1000)

In [3]:
recipe_df

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...,...
995,995,Winter of Our Content,"['3/4 ounce Laird’s Straight Apple Brandy', '3...","Combine the brandy, lemon juice, maple syrup, ...",winter-of-our-content-apple-brandy-sparkling-w...,"['3/4 ounce Laird’s Straight Apple Brandy', '3..."
996,996,Perfect Circle,['1 1/2 cups chilled fino or manzanilla sherry...,"Up to 24 hours before serving, make the batch....",perfect-circle-champagne-sherry-campari-cocktail,['1 1/2 cups chilled fino or manzanilla sherry...
997,997,All She Wrote,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ...","At least 2 hours before serving, make the batc...",all-she-wrote-punt-e-mes-vermouth-cocktail,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ..."
998,998,Mr. Tingles' Punch,"['1 (750 ml) bottle light rum', '2 tablespoons...",At least 24 hours before you plan to serve the...,mr-tingles-pomegranate-rum-punch,"['1 (750 ml) bottle light rum', '2 tablespoons..."


In [4]:
recipe_df = recipe_df.drop(columns=["Unnamed: 0"])
recipe_df

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...
995,Winter of Our Content,"['3/4 ounce Laird’s Straight Apple Brandy', '3...","Combine the brandy, lemon juice, maple syrup, ...",winter-of-our-content-apple-brandy-sparkling-w...,"['3/4 ounce Laird’s Straight Apple Brandy', '3..."
996,Perfect Circle,['1 1/2 cups chilled fino or manzanilla sherry...,"Up to 24 hours before serving, make the batch....",perfect-circle-champagne-sherry-campari-cocktail,['1 1/2 cups chilled fino or manzanilla sherry...
997,All She Wrote,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ...","At least 2 hours before serving, make the batc...",all-she-wrote-punt-e-mes-vermouth-cocktail,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ..."
998,Mr. Tingles' Punch,"['1 (750 ml) bottle light rum', '2 tablespoons...",At least 24 hours before you plan to serve the...,mr-tingles-pomegranate-rum-punch,"['1 (750 ml) bottle light rum', '2 tablespoons..."


In [5]:
print(recipe_df["Cleaned_Ingredients"].head(5).to_string(index=False))

['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher ...
['2 large egg whites', '1 pound new potatoes (a...
['1 cup evaporated milk', '1 cup whole milk', '...
['1 (¾- to 1-pound) round Italian loaf, cut int...
['1 teaspoon dark brown sugar', '1 teaspoon hot...


In [6]:
def clean_and_parse_list_str(x):
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
            else:
                return [str(parsed)]
        except (ValueError, SyntaxError):
            return [x]
    elif isinstance(x, list):
        return x
    else:
        return []

def clean_and_parse_ingredients(ingredient_str, full_clean=False):
    garbage_words = {
        "about", "a", "additional", "allpurpose", "and", "andor", "are", "as",
        "at", "but", "bunch", "bunches", "can", "chop", "cut", "each", "finely",
        "for", "for serving", "forget", "good", "goodquality", "great", "halved",
        "if", "i", "into", "lengthwise", "like", "melted", "minced", "mixed",
        "milliliters", "mild", "more", "new", "not", "of", "on", "on top", "optional",
        "or", "other", "package", "palmsize", "pinch", "pieces", "prefereably",
        "prepared", "really", "remove", "removed", "rim", "room temperature",
        "separate", "serving", "some", "small", "snapped", "snipped", "split",
        "sprig", "sprigs", "tablespoon", "tablespoons", "teaspoon", "teaspoons",
        "then", "the", "thinly", "this", "torn", "total", "to", "tq", "twist",
        "untied", "untrimmed", "use", "version", "very", "you", "qt", "quickcooking",
        "quarterred", "quality", "like", "knob", "handful", "needed", "pieces", "pound",
        "pounds", "ounce", "ounces", "large", "small", "each", "chilled", "a", "about",
        "plus", "divided", "such"
    }

    if isinstance(ingredient_str, str):
        try:
            parsed = ast.literal_eval(ingredient_str)
            if not isinstance(parsed, list):
                parsed = [str(parsed)]
        except (ValueError, SyntaxError):
            parsed = [ingredient_str]
    elif isinstance(ingredient_str, list):
        parsed = ingredient_str
    else:
        parsed = []

    if not full_clean:
        cleaned = []
        for item in parsed:
            item_clean = item.lower()
            for word in garbage_words:
                pattern = rf'\b{word}\b'
                item_clean = re.sub(pattern, '', item_clean)
            item_clean = re.sub(r'\s+', ' ', item_clean).strip()
            cleaned.append(item_clean)
        return cleaned
    else:
        cleaned = []
        for item in parsed:
            item = item.lower()
            item = re.sub(r'\band\b', ',', item)
            item = item.strip(' ,')
            parts = [p.strip() for p in item.split(',') if p.strip()]

            for p in parts:
                p = re.sub(r'[\d¼½¾⅓⅔⅛⅜⅝⅞\(\)\.\–\-]+', '', p)
                p = re.sub(r'\b(tsp|tbsp|cups?|oz|lb|lbs|g|kg|ml|l|inch|inches)\b', '', p, flags=re.IGNORECASE)

                for word in garbage_words:
                    p = re.sub(rf'\b{word}\b', '', p, flags=re.IGNORECASE)

                p = re.sub(r'[^a-zA-Z\s]', '', p)
                p = re.sub(r'\s+', ' ', p).strip()

                if p:
                    cleaned.append(p)
        return cleaned

# Proses data
recipe_df["Ingredients"] = recipe_df["Ingredients"].apply(clean_and_parse_list_str)
recipe_df["Ingredients"] = recipe_df["Ingredients"].apply(lambda x: clean_and_parse_ingredients(x, full_clean=False))
recipe_df["Cleaned_Ingredients"] = recipe_df["Ingredients"].apply(lambda x: clean_and_parse_ingredients(x, full_clean=True))

# Tambah food_id dan reorder kolom
recipe_df["food_id"] = recipe_df.apply(lambda _: uuid.uuid4().hex, axis=1)
cols = ["food_id"] + [col for col in recipe_df.columns if col != "food_id"]
recipe_df = recipe_df[cols]


In [7]:
recipe_df

,food_id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,fb89d512ef0443f8a9cfcacc68eca4c8,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"[whole chicken, kosher salt, acorn squash, cho..."
1,384bc4810e9941fe819c45633ec0c972,Crispy Salt and Pepper Potatoes,"[2 egg whites, 1 potatoes ( 1 inch in diameter...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"[egg whites, potatoes in diameter, kosher salt..."
2,e1f0f2e6af4d42429b207e3cf83249e8,Thanksgiving Mac and Cheese,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"[evaporated milk, whole milk, garlic powder, o..."
3,31afdefb75a34440b376cea81934c9a8,Italian Sausage and Bread Stuffing,"[1 (¾- 1-) round italian loaf, 1-inch cubes (8...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"[round italian loaf, cubes, olive oil, sweet i..."
4,2abc240176974ca78343853919f8237b,Newton's Law,"[1 dark brown sugar, 1 hot water, 1 ½ oz. bour...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"[dark brown sugar, hot water, bourbon, fresh l..."
...,...,...,...,...,...,...
995,a77e7e7caa21434b85fa4ea236cbf4eb,Winter of Our Content,"[3/4 laird’s straight apple brandy, 3/4 lemon ...","Combine the brandy, lemon juice, maple syrup, ...",winter-of-our-content-apple-brandy-sparkling-w...,"[lairds straight apple brandy, lemon juice, ma..."
996,d9315bdb5f2a489192dbecd6b0bef088,Perfect Circle,"[1 1/2 cups fino manzanilla sherry, 1 1/2 cups...","Up to 24 hours before serving, make the batch....",perfect-circle-champagne-sherry-campari-cocktail,"[fino manzanilla sherry, campari, bottle dry s..."
997,abdf05b6ff3f48fc9322ece01ebd0113,All She Wrote,"[2 1/4 cups punt e mes, 1 cup 2 dry vermouth (...","At least 2 hours before serving, make the batc...",all-she-wrote-punt-e-mes-vermouth-cocktail,"[punt e mes, dry vermouth dolin, maraschino li..."
998,1c514ab883554dd2bdf84350a7db08b8,Mr. Tingles' Punch,"[1 (750 ml) bottle light rum, 2 sichuan pepper...",At least 24 hours before you plan to serve the...,mr-tingles-pomegranate-rum-punch,"[bottle light rum, sichuan peppercorns, pomegr..."


In [8]:
recipe_df.to_csv("Food_Cleaned.csv")

In [9]:
recipe_df

,food_id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,fb89d512ef0443f8a9cfcacc68eca4c8,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"[whole chicken, kosher salt, acorn squash, cho..."
1,384bc4810e9941fe819c45633ec0c972,Crispy Salt and Pepper Potatoes,"[2 egg whites, 1 potatoes ( 1 inch in diameter...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"[egg whites, potatoes in diameter, kosher salt..."
2,e1f0f2e6af4d42429b207e3cf83249e8,Thanksgiving Mac and Cheese,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"[evaporated milk, whole milk, garlic powder, o..."
3,31afdefb75a34440b376cea81934c9a8,Italian Sausage and Bread Stuffing,"[1 (¾- 1-) round italian loaf, 1-inch cubes (8...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"[round italian loaf, cubes, olive oil, sweet i..."
4,2abc240176974ca78343853919f8237b,Newton's Law,"[1 dark brown sugar, 1 hot water, 1 ½ oz. bour...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"[dark brown sugar, hot water, bourbon, fresh l..."
...,...,...,...,...,...,...
995,a77e7e7caa21434b85fa4ea236cbf4eb,Winter of Our Content,"[3/4 laird’s straight apple brandy, 3/4 lemon ...","Combine the brandy, lemon juice, maple syrup, ...",winter-of-our-content-apple-brandy-sparkling-w...,"[lairds straight apple brandy, lemon juice, ma..."
996,d9315bdb5f2a489192dbecd6b0bef088,Perfect Circle,"[1 1/2 cups fino manzanilla sherry, 1 1/2 cups...","Up to 24 hours before serving, make the batch....",perfect-circle-champagne-sherry-campari-cocktail,"[fino manzanilla sherry, campari, bottle dry s..."
997,abdf05b6ff3f48fc9322ece01ebd0113,All She Wrote,"[2 1/4 cups punt e mes, 1 cup 2 dry vermouth (...","At least 2 hours before serving, make the batc...",all-she-wrote-punt-e-mes-vermouth-cocktail,"[punt e mes, dry vermouth dolin, maraschino li..."
998,1c514ab883554dd2bdf84350a7db08b8,Mr. Tingles' Punch,"[1 (750 ml) bottle light rum, 2 sichuan pepper...",At least 24 hours before you plan to serve the...,mr-tingles-pomegranate-rum-punch,"[bottle light rum, sichuan peppercorns, pomegr..."


In [10]:
# Buat mapping title ke cleaned_ingredients dan food_id
title_to_ingredients = dict(zip(recipe_df["Title"], recipe_df["Cleaned_Ingredients"]))
title_to_food_id = dict(zip(recipe_df["Title"], recipe_df["food_id"]))

# Buat user dan resep
recipes = recipe_df["Title"].unique()
num_users = 1000
user_ids = [f"user_{i+1}" for i in range(num_users)]

# Buat interactions
interactions = []
np.random.seed(42)

for user in user_ids:
    liked = np.random.choice(recipes, size=50, replace=False)
    for recipe in liked:
        ingredients = title_to_ingredients.get(recipe, [])
        food_id = title_to_food_id.get(recipe, None)
        interactions.append({
            "user_id": user,
            "food_id": food_id,
            "title": recipe,
            "ingredient": ingredients
        })

# Buat DataFrame
interaction_df = pd.DataFrame(interactions)

# Atur ulang seed untuk rating
np.random.seed(42)

# Jumlah total interaksi
n = len(interaction_df)

# Buat 90% rating > 3 (yaitu: 4 atau 5)
high_ratings = np.random.randint(4, 6, size=int(n * 0.9))
# Buat 10% rating <= 3
low_ratings = np.random.randint(1, 4, size=n - len(high_ratings))

# Gabungkan dan acak
all_ratings = np.concatenate([high_ratings, low_ratings])
np.random.shuffle(all_ratings)

# Masukkan ke DataFrame
interaction_df["rating"] = all_ratings

# Tambahkan kolom ingredient_str gabungan bahan dalam lowercase
interaction_df["ingredient_str"] = interaction_df["ingredient"].apply(
    lambda x: ' '.join([item.lower() for item in x]) if isinstance(x, list) else ''
)

# Tampilkan contoh
interaction_df

,user_id,food_id,title,ingredient,rating,ingredient_str
0,user_1,27ff62a3b1524e628fb18700f945c503,Pork Katsu Sandwich,"[boneless pork loin, fat trimmed thick, shio k...",1,boneless pork loin fat trimmed thick shio koji...
1,user_1,06840455c841427e80e62c66408073dc,Ice Water Salad,"[crunchy vegetables fennel, radishes, turnips,...",4,crunchy vegetables fennel radishes turnips bee...
2,user_1,2ee0f435ede34a1392cfe4b62bd1c133,Skirt Steak with Spicy Coconut Dressing,"[red jalapeno, fresno serrano chile, grated, g...",5,red jalapeno fresno serrano chile grated garli...
3,user_1,82243da87977412d96bdb7e6b9676450,Tomato-Watermelon Salad with Turmeric Oil,"[virgin coconut oil, coarsely crushed pepperco...",4,virgin coconut oil coarsely crushed peppercorn...
4,user_1,c08bf8b854ca44ec837670b7da97c2dd,Shredded Daikon Salad,"[daikon, peeled, sliced long matchsticks, kosh...",5,daikon peeled sliced long matchsticks kosher s...
...,...,...,...,...,...,...
49995,user_1000,9b7d9308e7fc4fb78141b37b519ce13f,Shaved Carrots with Charred Dates,"[purple orange carrots, trimmed, scrubbed, sha...",4,purple orange carrots trimmed scrubbed shaved ...
49996,user_1000,0f5b4e67948f4804ae82d0cf311c57de,"Sumac, Spelt, and Apple Cake","[granny smith bramley apples, peeled, cored ro...",4,granny smith bramley apples peeled cored rough...
49997,user_1000,7367c24cc0884da5afca9dda6cbd5272,Coconut Rice Pudding Pie,"[sesame sticks, granulated sugar, unsalted but...",1,sesame sticks granulated sugar unsalted butter...
49998,user_1000,a19ea965cf8e47a8a025d43127df3203,Sloppy Joe Shirred Eggs With Spinach,"[medium carrot, top, diced tomatoes, scant rou...",2,medium carrot top diced tomatoes scant roughly...


### Copy Image Data

In [11]:
# Path folder image
csv_path = "Food_Cleaned.csv"
source_dir = "Food Images/Food Images"
target_dir = "Images"

# Buat folder tujuan
os.makedirs(target_dir, exist_ok=True)

# Baca nama gambar
df = pd.read_csv(csv_path)
image_names = set(df["Image_Name"].dropna().astype(str).str.strip().str.lower())

# Ambil file yang tersedia
available_files = {f.lower() for f in os.listdir(source_dir)}
copied = 0
missing = []

# Salin file
for name in image_names:
    matched_file = None

    if name in available_files:
        matched_file = name
    else:
        for ext in [".jpg", ".jpeg", ".png"]:
            if name + ext in available_files:
                matched_file = name + ext
                break

    if matched_file:
        src_path = os.path.join(source_dir, matched_file)
        dst_path = os.path.join(target_dir, matched_file)
        shutil.copy2(src_path, dst_path)
        copied += 1
    else:
        missing.append(name)

# Cek
print(f"{copied} file berhasil disalin ke '{target_dir}'.")

if missing:
    print(f"{len(missing)} file tidak ditemukan, contoh:")
    print(missing[:10])

1000 file berhasil disalin ke 'Images'.


### Model Recommender

In [12]:
# ====================
# Data Preprocessing
# ====================
interaction_df["user_id"] = interaction_df["user_id"].astype(str)
interaction_df["food_id"] = interaction_df["food_id"].astype(str)
interaction_df["title"] = interaction_df["title"].astype(str)
interaction_df["ingredient_str"] = interaction_df["ingredient_str"].astype(str)

unique_user_ids = interaction_df["user_id"].unique()
unique_titles = interaction_df["title"].unique()

train_df, test_df = train_test_split(interaction_df, test_size=0.2, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": train_df["user_id"].values,
    "title": train_df["title"].values,
    "ingredient": train_df["ingredient_str"].values,
    "rating": train_df["rating"].values
}).shuffle(100).batch(128).cache()

val_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": test_df["user_id"].values,
    "title": test_df["title"].values,
    "ingredient": test_df["ingredient_str"].values,
    "rating": test_df["rating"].values
}).batch(64)

# ====================
# Lookup dan Text Vectorizer
# ====================
user_lookup = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
title_lookup = tf.keras.layers.StringLookup(vocabulary=unique_titles, mask_token=None)

text_vectorizer = tf.keras.layers.TextVectorization(
    output_mode="int",
    max_tokens=10000,
    output_sequence_length=50
)
text_vectorizer.adapt(interaction_df["ingredient_str"].values)

# ====================
# Arsitektur Model
# ====================
embedding_dim = 256

class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.model = tf.keras.Sequential([
            user_lookup,
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])

    def call(self, inputs):
        return self.model(inputs)

class ItemModel(tf.keras.Model):
    def __init__(self, text_vectorizer):
        super().__init__()
        self.title_model = tf.keras.Sequential([
            title_lookup,
            tf.keras.layers.Embedding(len(unique_titles) + 1, embedding_dim),
            tf.keras.layers.Dense(embedding_dim, activation="relu")
        ])
        self.ingredient_model = tf.keras.Sequential([
            text_vectorizer,
            tf.keras.layers.Embedding(text_vectorizer.vocabulary_size(), embedding_dim),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
            tf.keras.layers.Dense(embedding_dim, activation="relu")
        ])
        self.concat = tf.keras.layers.Concatenate()
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def call(self, features):
        title_embedding = self.title_model(features["title"])
        ingredient_embedding = self.ingredient_model(features["ingredient"])
        combined = self.concat([title_embedding, ingredient_embedding])
        return self.dense(combined)

class RecipeModel(tfrs.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model({
            "title": features["title"],
            "ingredient": features["ingredient"]
        })
        rating_predictions = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)
        return self.rating_task(labels=features["rating"], predictions=rating_predictions)

    def call(self, features):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model({
            "title": features["title"],
            "ingredient": features["ingredient"]
        })
        return tf.reduce_sum(user_embeddings * item_embeddings, axis=1)

# ====================
# Latih Model
# ====================
user_model = UserModel()
item_model = ItemModel(text_vectorizer)
model = RecipeModel(user_model, item_model)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0005))
early_stop = EarlyStopping(
    monitor="root_mean_squared_error",
    patience=10,
    restore_best_weights=True
)

model.fit(train_ds, validation_data=val_ds, epochs=300, callbacks=[early_stop])

# ====================
# Simpan Model
# ====================
os.makedirs("saved_model", exist_ok=True)

_ = model({
    "user_id": tf.constant(["dummy_user"]),
    "title": tf.constant(["dummy_title"]),
    "ingredient": tf.constant(["dummy ingredient"]),
    "rating": tf.constant([5.0])
})

user_model.save("saved_model/user_embedding_model")
item_model.save("saved_model/item_embedding_model")

# ====================
# BUat BruteForce Index
# ====================
unique_candidates = interaction_df.drop_duplicates(subset=["title"])[["title", "ingredient_str"]]

candidates_ds = tf.data.Dataset.from_tensor_slices({
    "title": unique_candidates["title"].values,
    "ingredient": unique_candidates["ingredient_str"].values
}).batch(128)

def map_fn(batch):
    embeddings = item_model({
        "title": batch["title"],
        "ingredient": batch["ingredient"]
    })
    return batch["title"], embeddings

candidates_ds = candidates_ds.map(map_fn)

index = tfrs.layers.factorized_top_k.BruteForce(user_model)
index.index_from_dataset(candidates_ds)

# ====================
# Simpan BruteForce Index
# ====================
class RecommendationIndex(tf.Module):
    def __init__(self, index_layer):
        super().__init__()
        self.index_layer = index_layer

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    ])
    def recommend(self, user_ids, k):
        return self.index_layer(user_ids, k=k)

recommender = RecommendationIndex(index)
tf.saved_model.save(recommender, "saved_model/brute_force_index")


Epoch 1/300
313/313 [==============================] - 19s 42ms/step - root_mean_squared_error: 1.2597 - loss: 1.5848 - regularization_loss: 0.0000e+00 - total_loss: 1.5848 - val_root_mean_squared_error: 0.9908 - val_loss: 1.0714 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0714
Epoch 2/300
313/313 [==============================] - 12s 38ms/step - root_mean_squared_error: 0.9401 - loss: 0.8843 - regularization_loss: 0.0000e+00 - total_loss: 0.8843 - val_root_mean_squared_error: 1.0041 - val_loss: 1.1765 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.1765
Epoch 3/300
313/313 [==============================] - 12s 38ms/step - root_mean_squared_error: 0.9110 - loss: 0.8305 - regularization_loss: 0.0000e+00 - total_loss: 0.8305 - val_root_mean_squared_error: 1.0160 - val_loss: 1.0841 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0841
Epoch 4/300
313/313 [==============================] - 12s 38ms/step - root_mean_squared_error: 0.8930 - loss: 0.7981 - r

INFO:tensorflow:Assets written to: saved_model/item_embedding_model\assets


INFO:tensorflow:Assets written to: saved_model/item_embedding_model\assets


INFO:tensorflow:Assets written to: saved_model/brute_force_index\assets


INFO:tensorflow:Assets written to: saved_model/brute_force_index\assets


#### Embedding

In [13]:
# Ambil data unik berdasarkan food_id supaya tidak ada duplikat
unique_candidates = interaction_df.drop_duplicates(subset=["food_id"])[["food_id", "title", "ingredient_str"]]

all_food_ids = unique_candidates["food_id"].tolist()
all_titles = unique_candidates["title"].tolist()
all_ingredients = unique_candidates["ingredient_str"].tolist()

food_ids_list = []
titles_list = []
embeddings_list = []

batch_size = 128
for i in range(0, len(all_titles), batch_size):
    batch_titles = tf.constant(all_titles[i:i+batch_size])
    batch_ingredients = tf.constant(all_ingredients[i:i+batch_size])
    batch_fids = all_food_ids[i:i+batch_size]

    # Hitung embedding item
    batch_embeddings = item_model({
        "title": batch_titles,
        "ingredient": batch_ingredients
    })

    # Simpan hasilnya ke list
    for fid, title, emb in zip(batch_fids, batch_titles.numpy(), batch_embeddings.numpy()):
        food_ids_list.append(fid)
        titles_list.append(title.decode("utf-8"))
        embeddings_list.append(emb)

# Simpan ke file .npz
np.savez(
    "saved_model/item_embeddings.npz",
    food_ids=np.array(food_ids_list),
    titles=np.array(titles_list),
    embeddings=np.array(embeddings_list)
)


#### Inference

In [14]:
# Buat mapping judul ke food_id
title_to_food_id = dict(zip(recipe_df["Title"], recipe_df["food_id"]))

user_id = "user_3"
user_interactions = interaction_df[interaction_df["user_id"] == user_id]

print(f"Resep yang telah diberi rating oleh user {user_id}:")
for _, row in user_interactions.iterrows():
    print(f"- {row['title']} (Rating: {row['rating']})")

visited_titles = set(user_interactions["title"].values)

# Panggil BruteForce index dengan input user_id
results = index(tf.constant([user_id]))
scores, titles = results

scores = scores.numpy()[0]
titles = titles.numpy()[0]

print(f"\nRekomendasi untuk user {user_id} (resep yang belum di-rate):")
count = 0
max_results = 20

for score, title in zip(scores, titles):
    title_str = title.decode("utf-8")
    if title_str in visited_titles:
        continue

    food_id = title_to_food_id.get(title_str)
    if food_id:
        print(f"- {title_str} (food_id: {food_id}, Skor: {score:.4f})")
        count += 1

    if count >= max_results:
        break

if count == 0:
    print("Tidak ada rekomendasi resep baru untuk ditampilkan.")
elif count < max_results:
    print(f"(Hanya {count} resep baru yang bisa direkomendasikan)")


Resep yang telah diberi rating oleh user user_3:
- New-and-Improved Greek Salad (Rating: 4)
- Squash and Radicchio Salad With Pecans (Rating: 4)
- Oaxaca Old Fashioned (Rating: 5)
- Za'atar Spice Blend (Rating: 5)
- Buttermilk Cornbread (Rating: 5)
- Butternut Squash, Coconut, and Ginger Muffins (Rating: 4)
- Tomato and Parmesan Risotto (Rating: 4)
- Camouflage Chocolate Fudge Brownies (Rating: 2)
- Fragrant Mixed Herb and Flatbread Salad (Domaaj) (Rating: 1)
- Tortillas de Harina (Flour Tortillas) (Rating: 5)
- Spaghetti with No-Cook Puttanesca (Rating: 5)
- Chopped BLT Salad (Rating: 4)
- Double Pecan Thumbprints (Rating: 4)
- Best Deviled Eggs (Rating: 4)
- Shrimp Ramp-y (Rating: 4)
- Jerk Baby Back Ribs With Pineapple Salsa (Rating: 4)
- Pork Volcánes al Pastor (Rating: 5)
- Instant Pot Lamb Haleem (Rating: 4)
- Torn Potatoes of Many Colors With Chile-Lime Butter (Rating: 5)
- Miso-Squash Ramen (Rating: 4)
- Big Green Lentil Salad (Rating: 3)
- Roasted Cauliflower with Parmesan-Pan

In [15]:
# Interaksi baru
new_interaction = {
    "user_id": "user_3",
    "food_id": "2169f474c62d40a2bf8ff005589db0e8",
    "title": "Cucumber-Seaweed Salad",
    "rating": 5,
    "ingredient": ["cucumber", "seaweed", "salad"]
}

# Tambah interaksi baru ke interaction_df
interaction_df = pd.concat([interaction_df, pd.DataFrame([new_interaction])], ignore_index=True)

# Hitung ulang embedding user baru
new_user_embedding = user_model(tf.constant([new_interaction["user_id"]])).numpy()[0]

# Load item embeddings dari .npz
data = np.load("saved_model/item_embeddings.npz", allow_pickle=True)
food_ids_np = data["food_ids"]
titles_np = data["titles"]
embeddings_np = data["embeddings"]

# Buat mapping untuk title ke food_id dari recipe_df (lowercase + strip)
title_to_food_id = dict(zip(recipe_df["Title"].str.lower().str.strip(), recipe_df["food_id"]))

# Buat mapping dari title_clean ke original title
title_clean_to_original = {}
for title in titles_np:
    if isinstance(title, bytes):
        original = title.decode("utf-8").strip()
    else:
        original = title.strip()
    clean = original.lower()
    if clean not in title_clean_to_original:
        title_clean_to_original[clean] = original

# Normalisasi casing titles dari embeddings dan buat dict unik
unique_items = {}
for fid, title, embedding in zip(food_ids_np, titles_np, embeddings_np):
    title_clean = title.decode("utf-8").lower().strip() if isinstance(title, bytes) else title.lower().strip()
    if title_clean not in unique_items:
        unique_items[title_clean] = (fid, embedding)

# Kumpulkan judul yang sudah pernah di-rating user (lowercase + strip)
visited_titles = set(interaction_df[interaction_df["user_id"] == new_interaction["user_id"]]["title"].str.lower().str.strip())

# Hitung skor rekomendasi
results = []
for title_clean, (fid, embedding) in unique_items.items():
    if title_clean in visited_titles:
        continue
    score = np.dot(new_user_embedding, embedding)
    results.append((title_clean, fid, score))

# Urutkan dan tampilkan hasil rekomendasi dengan judul asli
results.sort(key=lambda x: x[2], reverse=True)

print(f"Rekomendasi baru untuk user {new_interaction['user_id']}:")
for title_clean, fid, score in results[:10]:
    original_title = title_clean_to_original.get(title_clean, title_clean)
    print(f"- {original_title} (food_id: {fid}, Skor: {score:.4f})")

Rekomendasi baru untuk user user_3:
- Double-Fried Chicken Wings (food_id: b591cf0fc8d94db5a987bef2b97ab16f, Skor: 6.0197)
- Grilled Lamb Chops with Kefir Verde Sauce (food_id: 751ca06d91d64dd994824aabc64e0f61, Skor: 5.7970)
- Winter Salad Hummus Bowls (food_id: f6c40ebf1d894974ad9097eb4162f490, Skor: 5.7710)
- Tiger Fruit Salad (food_id: b588d16369ec4c2d96705022b8b97756, Skor: 5.7571)
- Long-Roasted Eggplant with Garlic, Labne, and Tiny Chile Croutons (food_id: 24946b91ccd742b28e3db22815a99f97, Skor: 5.7495)
- Molasses Sandwich Cookies with Coffee Cream (food_id: 2f7d898d793846db87cc15170ae1df11, Skor: 5.6334)
- Trout Toast with Soft Scrambled Eggs (food_id: d437b364b0f14d2ea7cc4fe68e239ac8, Skor: 5.5849)
- Grilled Potatoes With Red Miso Butter (food_id: ee44ca00a3f348d88e2dbe6f5be15c7e, Skor: 5.5545)
- Mala Fried Peanuts (food_id: cc2bcf07708b47238d780eeda7979e6f, Skor: 5.5430)
- Tomato Galette (food_id: f926983243524a0e9a5b638448b0eaa5, Skor: 5.5161)


#### Evaluasi Model

In [16]:
# ====================
# Data dan variabel awal
# ====================
item_popularity = Counter(interaction_df["title"])
total_interaction = sum(item_popularity.values())
all_items = set(interaction_df["title"].unique())

user_history = interaction_df.groupby("user_id")["title"].apply(set).to_dict()

# ====================
# Precision@K
# ====================
def precision_at_k(k):
    correct = 0
    total = 0
    test_dict = test_df.groupby("user_id")["title"].apply(set).to_dict()

    for user_id, true_items in test_dict.items():
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_ids = set(title.numpy().decode("utf-8") for title in recommended[0])
        hits = recommended_ids.intersection(true_items)
        correct += len(hits)
        total += k
    return correct / total if total > 0 else 0

# ====================
# Novelty@K
# ====================
def novelty_at_k(k):
    total_novelty = 0.0
    total_users = 0

    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        novelty_score = 0.0
        for title in recommended[0].numpy():
            title_str = title.decode("utf-8")
            freq = item_popularity[title_str] / total_interaction
            if freq > 0:
                novelty_score += -np.log2(freq)
        avg_novelty = novelty_score / k
        total_novelty += avg_novelty
        total_users += 1

    return total_novelty / total_users if total_users > 0 else 0

# ====================
# Coverage@K
# ====================
def coverage_at_k(k):
    recommended_items = set()
    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        for title in recommended[0].numpy():
            recommended_items.add(title.decode("utf-8"))
    return len(recommended_items) / len(all_items)

# ============================
# Pre-komputasi Serendipity
# ============================
title_to_embedding = {}
for title in interaction_df["title"].unique():
    features = {
        "title": tf.constant([title]),
        "ingredient": tf.constant([""])
    }
    vec = model.item_model(features)
    vec = tf.math.l2_normalize(vec, axis=1)
    title_to_embedding[title] = vec.numpy()[0]

user_mean_embeddings = {}
for user_id in test_df["user_id"].unique():
    seen = user_history.get(user_id, set())
    seen_vecs = [title_to_embedding[title] for title in seen if title in title_to_embedding]
    if not seen_vecs:
        continue
    seen_avg = np.mean(seen_vecs, axis=0)
    seen_avg = seen_avg / np.linalg.norm(seen_avg)
    user_mean_embeddings[user_id] = seen_avg

# ============================
# Serendipity@K
# ============================
def make_serendipity_function(k):
    def compute_serendipity_batch(user_id_tensor):
        user_id = user_id_tensor.numpy().decode("utf-8") if isinstance(user_id_tensor.numpy(), bytes) else str(user_id_tensor.numpy())

        seen = user_history.get(user_id, set())
        if not seen or user_id not in user_mean_embeddings:
            return tf.constant(0.0, dtype=tf.float32)

        seen_avg = user_mean_embeddings[user_id].reshape(1, -1)
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_titles = recommended[0].numpy()

        ser = 0
        valid_k = 0
        for title in recommended_titles:
            title_str = title.decode("utf-8")
            if title_str in seen or title_str not in title_to_embedding:
                continue
            rec_vec = title_to_embedding[title_str].reshape(1, -1)
            dot_sim = np.dot(seen_avg, rec_vec.T)[0][0]
            ser += 1 - dot_sim
            valid_k += 1

        score = ser / valid_k if valid_k > 0 else 0.0
        return tf.constant(score, dtype=tf.float32)

    return compute_serendipity_batch

def serendipity_at_k(k):
    compute_fn = make_serendipity_function(k)
    user_ids_tensor = tf.constant(list(user_mean_embeddings.keys()), dtype=tf.string)

    serendipity_scores = tf.data.Dataset.from_tensor_slices(user_ids_tensor) \
        .map(lambda uid: tf.py_function(func=compute_fn, inp=[uid], Tout=tf.float32)) \
        .batch(128) \
        .map(lambda batch: tf.reduce_sum(batch)) \
        .reduce(tf.constant(0.0), lambda acc, val: acc + val)

    return serendipity_scores.numpy() / len(user_mean_embeddings)

# ============================
# Evaluasi Semua Metrik
# ============================
def evaluate_model(k_values=[10]):
    print("Evaluasi sistem rekomendasi:")
    for k in k_values:
        print(f"\nTop-{k}")
        precision = precision_at_k(k)
        novelty = novelty_at_k(k)
        coverage = coverage_at_k(k)
        serendipity = serendipity_at_k(k)
        print(f"- Precision@{k}:   {precision:.4f}")
        print(f"- Novelty@{k}:     {novelty:.4f}")
        print(f"- Coverage@{k}:    {coverage:.4f}")
        print(f"- Serendipity@{k}: {serendipity:.4f}")

# ============================
# Jalankan Evaluasi
# ============================
evaluate_model(k_values=[10, 50, 100])

Evaluasi sistem rekomendasi:

Top-10
- Precision@10:   0.0104
- Novelty@10:     9.9612
- Coverage@10:    0.9060
- Serendipity@10: 0.4063

Top-50
- Precision@50:   0.0106
- Novelty@50:     9.9700
- Coverage@50:    0.9980
- Serendipity@50: 0.3943

Top-100
- Precision@100:   0.0100
- Novelty@100:     9.9720
- Coverage@100:    1.0000
- Serendipity@100: 0.3889
